Tutorial on Federated Learning with FedAvg using PyTorch and PySyft based on https://learnopencv.com/federated-learning-using-pytorch-and-pysyft/ and https://towardsdatascience.com/federated-learning-a-simple-implementation-of-fedavg-federated-averaging-with-pytorch-90187c9c9577

## Install PySyft and Torchvision

In [ ]:
!pip install syft==0.2.9

In [ ]:
!pip install torchvision==0.2.0

## Simple Testing on PySyft (can be skipped)

In [ ]:
import torch
import syft as sy
hook = sy.TorchHook(torch) # add extra functionality to PyTorch

In [ ]:
# create a machine owned by harmony clinic
harmony_clinic = sy.VirtualWorker(hook=hook,id='clinic')

In [ ]:
# we create a Tensor, maybe this is some gene sequence
dna = torch.tensor([0,1,2,1,2])

# and now I send it, and in turn we get a pointer back that
# points to that Tensor
dna_ptr = dna.send(harmony_clinic)

print(dna_ptr)
print(harmony_clinic._objects)

In [ ]:
# get back dna
dna = dna_ptr.get()
print(dna)

# And as you can see... clinic no longer has the tensor dna anymore!!! It has moved back to our machine!
print(harmony_clinic._objects)

tensor([0, 1, 2, 1, 2])
{}


In [ ]:
# we create two tensors and send them to clinic
train = torch.tensor([2.4, 6.2], requires_grad=True).send(harmony_clinic)
label = torch.tensor([2, 6.]).send(harmony_clinic)

# we apply some function, in this case a rather simple one, just to show the idea, we use L1 loss
loss = (train-label).abs().sum()

# Yes, even .backward() works when working with Pointers
loss.backward()

# now we retreive back the train tensor
train = train.get()

print(train)

# If everything went well, we will see gradients accumulated 
# in .grad attribute of train
print(train.grad)

tensor([2.4000, 6.2000], requires_grad=True)
tensor([1., 1.])


## FL on MNIST dataset

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import logging

# import Pysyft to help us to simulate federated learning
import syft as sy

# hook PyTorch to PySyft, i.e. add extra functionalities to support Federated Learning and other private AI tools
hook = sy.TorchHook(torch) 

# create clients
clients = []
clients.append(sy.VirtualWorker(hook, id="bob"))
clients.append(sy.VirtualWorker(hook, id="alice"))
clients.append(sy.VirtualWorker(hook, id="untrustful"))

In [2]:
# define the args
args = {
    'use_cuda' : True,
    'batch_size' : 64,
    'test_batch_size' : 1000,
    'lr' : 0.01,
    'log_interval' : 10,
    # with federated learning convergence is faster and less epochs are needed
    'epochs' : 7
}

# check to use GPU or not
use_cuda = args['use_cuda'] and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [3]:
# create a simple CNN net
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels = 1, out_channels = 32, kernel_size = 3, stride = 1),
            nn.ReLU(),
            nn.Conv2d(in_channels=32,out_channels = 64, kernel_size = 3, stride = 1),
            nn.ReLU()
        )
        
        self.fc = nn.Sequential(
            nn.Linear(in_features=64*12*12, out_features=128),
            nn.ReLU(),
            nn.Linear(in_features=128, out_features=10),
        )

        self.dropout = nn.Dropout2d(0.25)
    
    def forward(self, x):
        x = self.conv(x)
        x = F.max_pool2d(x,2)
        x = x.view(-1, 64*12*12)
        x = self.fc(x)
        x = F.log_softmax(x, dim=1)
        return x

In [ ]:
# prepare and distribute the data across workers
# normally there is no need to distribute data, since it is already at the clients
# this is more of a simulation of federated learning
federated_train_loader = sy.FederatedDataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
    .federate(tuple(clients)),
    batch_size=args['batch_size'], shuffle=True)

# test data remains at the central entity
test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=args['test_batch_size'], shuffle=True)

In [5]:
# classic torch code for training except for the federated part
def train_locally(args, models, device, train_loader, optimizers, epoch):
    for c, m in models.items():
        m.train()
        # send models to workers
        m.send(c)

    # iterate over federated data
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizers[data.location].zero_grad()
        output = models[data.location](data)
        # loss is a ptr to the tensor loss at the remote location
        loss = F.nll_loss(output, target)
        # call backward() on the loss ptr, that will send the command to call
        # backward on the actual loss tensor present on the remote machine
        loss.backward()
        optimizers[data.location].step()

        if batch_idx % args['log_interval'] == 0:

            # get back loss, that was created at remote worker
            loss = loss.get()

            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tWorker: {}'.format(
                    epoch, 
                    batch_idx * args['batch_size'], # no of images done
                    len(train_loader) * args['batch_size'], # total images left
                    100. * batch_idx / len(train_loader),
                    loss,
                    data.location.id
                )
            )
    
    # get back models for aggregation
    for m in models.values():
        m = m.get()

In [6]:
# classic torch code for testing
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)

            # add losses together
            test_loss += F.nll_loss(output, target, reduction='sum').item() 

            # get the index of the max probability class
            pred = output.argmax(dim=1, keepdim=True)  
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [7]:
def aggregate(central_model, models, weights):
    # firstly compute mean weight values
    with torch.no_grad():
        for m in models.values():
            weights['conv0_mean_weight'] += m.conv[0].weight.data.clone()
            weights['conv0_mean_bias'] += m.conv[0].bias.data.clone()
            weights['conv2_mean_weight'] += m.conv[2].weight.data.clone()
            weights['conv2_mean_bias'] += m.conv[2].bias.data.clone()            
            weights['fc0_mean_weight'] += m.fc[0].weight.data.clone()
            weights['fc0_mean_bias'] += m.fc[0].bias.data.clone()
            weights['fc2_mean_weight'] += m.fc[2].weight.data.clone()
            weights['fc2_mean_bias'] += m.fc[2].bias.data.clone()            

        weights['conv0_mean_weight'] = weights['conv0_mean_weight']/len(clients)
        weights['conv0_mean_bias'] = weights['conv0_mean_bias']/len(clients)
        weights['conv2_mean_weight'] = weights['conv2_mean_weight']/len(clients)
        weights['conv2_mean_bias'] = weights['conv2_mean_bias']/len(clients)
        weights['fc0_mean_weight'] = weights['fc0_mean_weight']/len(clients)
        weights['fc0_mean_bias'] = weights['fc0_mean_bias']/len(clients)
        weights['fc2_mean_weight'] = weights['fc2_mean_weight']/len(clients)
        weights['fc2_mean_bias'] = weights['fc2_mean_bias']/len(clients)

        # then copy them to the local models
        for m in models.values():
            m.conv[0].weight.data = weights['conv0_mean_weight'].data.clone()
            m.conv[0].bias.data = weights['conv0_mean_bias'].data.clone()
            m.conv[2].weight.data = weights['conv2_mean_weight'].data.clone()
            m.conv[2].bias.data = weights['conv2_mean_bias'].data.clone()
            m.fc[0].weight.data = weights['fc0_mean_weight'].data.clone()
            m.fc[0].bias.data = weights['fc0_mean_bias'].data.clone()
            m.fc[2].weight.data = weights['fc2_mean_weight'].data.clone()
            m.fc[2].bias.data = weights['fc2_mean_bias'].data.clone()

        # and to the central model for the test set
        central_model.conv[0].weight.data = weights['conv0_mean_weight'].data.clone()
        central_model.conv[0].bias.data = weights['conv0_mean_bias'].data.clone()
        central_model.conv[2].weight.data = weights['conv2_mean_weight'].data.clone()
        central_model.conv[2].bias.data = weights['conv2_mean_bias'].data.clone()
        central_model.fc[0].weight.data = weights['fc0_mean_weight'].data.clone()
        central_model.fc[0].bias.data = weights['fc0_mean_bias'].data.clone()
        central_model.fc[2].weight.data = weights['fc2_mean_weight'].data.clone()
        central_model.fc[2].bias.data = weights['fc2_mean_bias'].data.clone()

In [8]:
# central model
central_model = Net().to(device)
# optimizer for central model not needed if model is not to be trained
#optimizer = optim.SGD(central_model.parameters(), lr=args['lr'])
# clients models and optimizers
models = {i:Net().to(device) for i in clients}
optimizers = {i:optim.SGD(models[i].parameters(), lr=args['lr']) for i in clients}

# initialization of dictionary for model aggregation
weights = {'conv0_mean_weight' : torch.zeros(size=central_model.conv[0].weight.shape).to(device),
           'conv0_mean_bias' : torch.zeros(size=central_model.conv[0].bias.shape).to(device),
           'conv2_mean_weight' : torch.zeros(size=central_model.conv[2].weight.shape).to(device),
           'conv2_mean_bias' : torch.zeros(size=central_model.conv[2].bias.shape).to(device),
           'fc0_mean_weight' : torch.zeros(size=central_model.fc[0].weight.shape).to(device),
           'fc0_mean_bias' : torch.zeros(size=central_model.fc[0].bias.shape).to(device),
           'fc2_mean_weight' : torch.zeros(size=central_model.fc[2].weight.shape).to(device),
           'fc2_mean_bias' : torch.zeros(size=central_model.fc[2].bias.shape).to(device)}

logging.info("Starting training !!")

for epoch in range(1, args['epochs'] + 1):
    train_locally(args, models, device, federated_train_loader, optimizers, epoch)
    aggregate(central_model, models, weights)
    test(central_model, device, test_loader)

Train Epoch: 1 [0/60032 (0%)]	Loss: 2.282876	Worker: bob
Train Epoch: 1 [640/60032 (1%)]	Loss: 2.228942	Worker: bob
Train Epoch: 1 [1280/60032 (2%)]	Loss: 2.086825	Worker: bob
Train Epoch: 1 [1920/60032 (3%)]	Loss: 1.857036	Worker: bob
Train Epoch: 1 [2560/60032 (4%)]	Loss: 1.583934	Worker: bob
Train Epoch: 1 [3200/60032 (5%)]	Loss: 1.012585	Worker: bob
Train Epoch: 1 [3840/60032 (6%)]	Loss: 0.795606	Worker: bob
Train Epoch: 1 [4480/60032 (7%)]	Loss: 0.893307	Worker: bob
Train Epoch: 1 [5120/60032 (9%)]	Loss: 0.673929	Worker: bob
Train Epoch: 1 [5760/60032 (10%)]	Loss: 0.616753	Worker: bob
Train Epoch: 1 [6400/60032 (11%)]	Loss: 0.536093	Worker: bob
Train Epoch: 1 [7040/60032 (12%)]	Loss: 0.607093	Worker: bob
Train Epoch: 1 [7680/60032 (13%)]	Loss: 0.531591	Worker: bob
Train Epoch: 1 [8320/60032 (14%)]	Loss: 0.330176	Worker: bob
Train Epoch: 1 [8960/60032 (15%)]	Loss: 0.411866	Worker: bob
Train Epoch: 1 [9600/60032 (16%)]	Loss: 0.281410	Worker: bob
Train Epoch: 1 [10240/60032 (17%)]	Lo

In [11]:
# now we repeat training, but with one client being untrustful

# central model
central_model = Net().to(device)
# optimizer for central model not needed if model is not to be trained
#optimizer = optim.SGD(central_model.parameters(), lr=args['lr'])
# clients models and optimizers
models = {i:Net().to(device) for i in clients}
optimizers = {i:optim.SGD(models[i].parameters(), lr=args['lr']) for i in clients}

# initialization of dictionary for model aggregation
weights = {'conv0_mean_weight' : torch.zeros(size=central_model.conv[0].weight.shape).to(device),
           'conv0_mean_bias' : torch.zeros(size=central_model.conv[0].bias.shape).to(device),
           'conv2_mean_weight' : torch.zeros(size=central_model.conv[2].weight.shape).to(device),
           'conv2_mean_bias' : torch.zeros(size=central_model.conv[2].bias.shape).to(device),
           'fc0_mean_weight' : torch.zeros(size=central_model.fc[0].weight.shape).to(device),
           'fc0_mean_bias' : torch.zeros(size=central_model.fc[0].bias.shape).to(device),
           'fc2_mean_weight' : torch.zeros(size=central_model.fc[2].weight.shape).to(device),
           'fc2_mean_bias' : torch.zeros(size=central_model.fc[2].bias.shape).to(device)}

logging.info("Starting training !!")

for epoch in range(1, args['epochs'] + 1):
    train_locally(args, models, device, federated_train_loader, optimizers, epoch)
    # we intentionally change the weights of a client's model to make that client behave as untrustful
    models[clients[2]].conv[0].weight.data = models[clients[2]].conv[0].weight.data*10
    models[clients[2]].conv[2].weight.data = models[clients[2]].conv[2].weight.data*10
    models[clients[2]].fc[0].weight.data = models[clients[2]].fc[0].weight.data*10
    models[clients[2]].fc[2].weight.data = models[clients[2]].fc[2].weight.data*10
    aggregate(central_model, models, weights)
    test(central_model, device, test_loader)

Train Epoch: 1 [0/60032 (0%)]	Loss: 2.295660	Worker: bob
Train Epoch: 1 [640/60032 (1%)]	Loss: 2.162782	Worker: bob
Train Epoch: 1 [1280/60032 (2%)]	Loss: 1.972803	Worker: bob
Train Epoch: 1 [1920/60032 (3%)]	Loss: 1.656957	Worker: bob
Train Epoch: 1 [2560/60032 (4%)]	Loss: 1.294741	Worker: bob
Train Epoch: 1 [3200/60032 (5%)]	Loss: 0.995847	Worker: bob
Train Epoch: 1 [3840/60032 (6%)]	Loss: 0.958026	Worker: bob
Train Epoch: 1 [4480/60032 (7%)]	Loss: 0.586278	Worker: bob
Train Epoch: 1 [5120/60032 (9%)]	Loss: 0.562689	Worker: bob
Train Epoch: 1 [5760/60032 (10%)]	Loss: 0.673915	Worker: bob
Train Epoch: 1 [6400/60032 (11%)]	Loss: 0.283507	Worker: bob
Train Epoch: 1 [7040/60032 (12%)]	Loss: 0.527617	Worker: bob
Train Epoch: 1 [7680/60032 (13%)]	Loss: 0.477611	Worker: bob
Train Epoch: 1 [8320/60032 (14%)]	Loss: 0.405786	Worker: bob
Train Epoch: 1 [8960/60032 (15%)]	Loss: 0.481299	Worker: bob
Train Epoch: 1 [9600/60032 (16%)]	Loss: 0.316219	Worker: bob
Train Epoch: 1 [10240/60032 (17%)]	Lo